In [1]:
import pandas as pd

import geopandas as gpd

import requests

import datetime as dt
from pyathena import connect

In [39]:
import boto3

client = boto3.client('athena')

response = client.get_work_group(WorkGroup='primary')
print(response['WorkGroup']['Configuration']['EngineVersion'])

{'SelectedEngineVersion': 'AUTO', 'EffectiveEngineVersion': 'Athena engine version 3'}


```bash
gunzip -c station_1502.csv.gz | head -20
"location_id","sensors_id","location","datetime","lat","lon","parameter","units","value"
1502,2652,"Beaverton - Highland-1502","2025-10-01T01:00:00-07:00","45.4702","-122.8159","pm25","µg/m³","1.0"
1502,2652,"Beaverton - Highland-1502","2025-10-01T02:00:00-07:00","45.4702","-122.8159","pm25","µg/m³","1.0"
1502,2652,"Beaverton - Highland-1502","2025-10-01T03:00:00-07:00","45.4702","-122.8159","pm25","µg/m³","1.0"
```

https://docs.aws.amazon.com/athena/latest/ug/supported-serdes.html


```
CSV (Comma-Separated Values)
	

For data in CSV, each line represents a data record, and each record consists of one or more fields, separated by commas.
	

    Use the Lazy Simple SerDe for CSV, TSV, and custom-delimited files if your data does not include values enclosed in quotes or if it uses the java.sql.Timestamp format.

    Use the Open CSV SerDe for processing CSV when your data includes quotes in values or uses the UNIX numeric format for TIMESTAMP (for example, 1564610311).
```

https://docs.aws.amazon.com/athena/latest/ug/csv-serde.html

In [2]:
cursor = connect(
    s3_staging_dir="s3://la-openaq-athena-results-us-east-1/temp/",
    region_name="us-east-1"
).cursor()

In [ ]:
Create_query = """
CREATE EXTERNAL TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
PARTITIONED BY (locationid STRING, year STRING, month STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format'=',','field.delim'=',')
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar"     = "`",
  "escapeChar"    = "\\"
)
"""

In [1]:
Create_Table_Command = """
CREATE TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"',
   'escapeChar' = '\\'
   )
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor.execute(Create_Table_Command)

NameError: name 'cursor' is not defined

In [8]:
Create_Table_Command = """
CREATE TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
WITH (format = 'CSV', external_location = 's3://...')
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor.execute(Create_Table_Command)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 13:1: mismatched input 'STORED'. Expecting: '%', ')', '*', '+', ',', '-', '.', '/', 'AND', 'AT', 'OR', '[', '||', <EOF>, <predicate>

https://trino.io/docs/current/connector/hive.html

cursor.execute('DROP TABLE openaq_db.openaqMeasurements')

In [21]:
trino_query = """
CREATE TABLE openaq_db.openaqMeasurements (
  location_id INT
)
WITH (
  format = 'TEXTFILE',
  external_location = 's3://openaq-data-archive/records/csv.gz/'
)
"""
cursor.execute(trino_query)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 4:6: no viable alternative at input 'CREATE TABLE openaq_db.openaqMeasurements (\n  location_id INT\n)\nWITH ('

In [11]:
trino_query2 = """
CREATE TABLE openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
WITH (
  format = 'CSV',
  external_location = 's3://openaq-data-archive/records/csv.gz/'
)
"""
cursor.execute(trino_query2)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 12:6: no viable alternative at input 'CREATE TABLE openaq_db.openaqMeasurements (\n  location_id INT,\n  sensors_id INT,\n  location STRING,\n  datetime STRING,\n  lat DOUBLE,\n  lon DOUBLE,\n  parameter STRING,\n  units STRING,\n  value DOUBLE\n)\nWITH ('

In [16]:
query = """
CREATE EXTERNAL TABLE openaq_db.openaqMeasurements (
   col1 string,
   col2 string,
   col3 string,
   col4 string
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   'separatorChar' = ',',
   'quoteChar' = '"',
   'escapeChar' = '\\'
   )
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/';
"""

cursor.execute(query)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'MATERIALIZED', 'MULTI', 'OR', 'PROTECTED', 'ROLE', 'SCHEMA', 'TABLE', 'VIEW'

In [ ]:
cursor.execute('DROP TABLE 

In [19]:
cursor.execute("""
CREATE EXTERNAL TABLE test1 (
f1 string,
s2 string) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
WITH SERDEPROPERTIES ("separatorChar" = ",", "escapeChar" = "\\") 
LOCATION 's3://amzn-s3-demo-bucket/dataset/test1/'
"""
)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'MATERIALIZED', 'MULTI', 'OR', 'PROTECTED', 'ROLE', 'SCHEMA', 'TABLE', 'VIEW'

In [22]:
test3 = """
CREATE EXTERNAL TABLE my_table
    (
        ID string,
        PERSON_ID int,
        DATE_COL date,
        GMAT int
    )
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES
    (
        'skip.header.line.count'='1'
    )
;
"""


cursor.execute(test3)

In [41]:
cursor.execute("DROP TABLE IF EXISTS openaqMeasurements")

test3 = """
CREATE EXTERNAL TABLE `openaqMeasurements`(
  `location_id` INT,
  `sensors_id` INT,
  `location` STRING,
  `datetime` STRING,
  `lat` float,
  `lon` float,
  `parameter` STRING,
  `units` STRING,
  `value` float
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  "separatorChar" = ",",
  "quoteChar" = "\"",
  "escapeChar" = "\\"
)
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES
    (
        'skip.header.line.count'='1'
    )
;
"""

cursor.execute(test3)

Failed to execute query.
Traceback (most recent call last):
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/common.py", line 645, in _execute
    query_id = retry_api_call(
               ~~~~~~~~~~~~~~^
        self._connection.client.start_query_execution,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<2 lines>...
        **request,
        ^^^^^^^^^^
    ).get("QueryExecutionId")
    ^
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/pyathena/util.py", line 196, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
  File "/home/daniel/miniforge3/envs/la_env/lib/python3.14/site-packages/tenacity/__init_

DatabaseError: An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'EXTERNAL'. Expecting: 'MATERIALIZED', 'MULTI', 'OR', 'PROTECTED', 'ROLE', 'SCHEMA', 'TABLE', 'VIEW'

In [27]:
cursor.execute("DROP TABLE openaq_db.openaqMeasurements")

In [28]:
test3 = """
CREATE EXTERNAL TABLE `openaq_db.openaqMeasurements`(
  `location_id` INT,
  `sensors_id` INT,
  `location` STRING,
  `datetime` STRING,
  `lat` float,
  `lon` float,
  `parameter` STRING,
  `units` STRING,
  `value` float
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
STORED AS TEXTFILE
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES
    (
        'skip.header.line.count'='1'
    )
;
"""

cursor.execute(test3)

In [29]:
location = 1200

In [30]:
cursor.execute(f"""
ALTER TABLE openaq_db.openaqMeasurements ADD
PARTITION (year='2025', month='09', locationid='{location}')
LOCATION 's3://openaq-data-archive/records/csv.gz/locationid={location}/year=2025/month=09/';
""")

OperationalError: FAILED: SemanticException table is not partitioned but partition spec exists: {year=2025, month=09, locationid=1200}

In [32]:
cursor.execute('MSCK REPAIR TABLE openaq_db.openaqMeasurements;')

In [33]:
cursor.execute("SHOW PARTITIONS openaq_db.openaqMeasurements;")
print(cursor.fetchall())

OperationalError: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. Table openaq_db.openaqMeasurements is not a partitioned table

In [34]:
cursor.execute("SELECT * FROM openaq_db.openaqMeasurements LIMIT 1;")
print(cursor.fetchall())

[(1, 1, 'JTC - Jamestown-1', '2006-11-19T07:55:00+00:00', 5.5349193, -0.2124056, 'pm10', 'µg/m³', 218.14)]


In [4]:
cursor.execute("DROP TABLE IF EXISTS openaq_db.openaqMeasurements")

test3 = """
CREATE EXTERNAL TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
PARTITIONED BY (locationid STRING, year STRING, month STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('serialization.format'=',','field.delim'=',')
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""
cursor.execute(test3)

In [7]:
recents_pm25 = gpd.read_file(r'../Data/Outputs/recents_pm25.geojson')

In [8]:
recents_pm25.iloc[0]

station_id                                            1200
name                                     Glendora - Laurel
sensor_id                                             2150
sensor_display_name                                  PM2.5
timezone                               America/Los_Angeles
is_mobile                                            False
is_monitor                                            True
owner.id                                                 4
owner.name               Unknown Governmental Organization
provider.id                                            119
provider.name                                       AirNow
coordinates.latitude                               34.1439
coordinates.longitude                            -117.8508
datetime_first.utc               2016-03-06 20:00:00+00:00
datetime_last.utc                2025-10-24 19:00:00+00:00
sensor_type                                     pm25 µg/m³
geometry                         POINT (34.1439 -117.850

In [18]:
location = 1200

In [19]:
cursor.execute(f"""
ALTER TABLE openaq_db.openaqMeasurements ADD
PARTITION (year='2025', month='09', locationid='{location}')
LOCATION 's3://openaq-data-archive/records/csv.gz/locationid={location}/year=2025/month=09/';
""")

In [20]:
cursor.execute("SHOW PARTITIONS openaq_db.openaqMeasurements;")
print(cursor.fetchall())

[('locationid=2178/year=2022/month=5',), ('locationid=1200/year=2025/month=09',)]


In [21]:
cursor.execute("SELECT * FROM openaq_db.openaqMeasurements LIMIT 1;")
print(cursor.fetchall())

[(1200, 2152, '"Glendora - Laurel-1200"', '"2025-09-15T01:00:00-07:00"', None, None, '"pm10"', '"µg/m³"', None, '1200', '2025', '09')]


-----

In [17]:
cursor.execute("DROP TABLE IF EXISTS openaq_db.openaqMeasurements")

create_table_query = """
CREATE EXTERNAL TABLE IF NOT EXISTS openaq_db.openaqMeasurements (
  location_id INT,
  sensors_id INT,
  location STRING,
  datetime STRING,
  lat DOUBLE,
  lon DOUBLE,
  parameter STRING,
  units STRING,
  value DOUBLE
)
PARTITIONED BY (locationid STRING, year STRING, month STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format'=',','field.delim'=',')
LOCATION 's3://openaq-data-archive/records/csv.gz/'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

cursor.execute(create_table_query)

year = 2022
month = 5
location = 2178

cursor.execute(f"""
ALTER TABLE openaq_db.openaqMeasurements ADD
PARTITION (year='{year}', month='{month}', locationid='{location}')
LOCATION 's3://openaq-data-archive/records/csv.gz/locationid={location}/year={year}/month={month}/';
""")

cursor.execute("SELECT * FROM openaq_db.openaqMeasurements LIMIT 1;")
print(cursor.fetchall())

[]


` /records/csv.gz/locationid=2178/year=2022/month=05/location-2178-20220503.csv.gz `